In [1]:
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '..')

# for saving
import os

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from src import setup
# setup.use_gpu()

In [2]:
from src import ( 
    measurements as measure,
    density_matrix as DM,
    simulation as sim,
    orders,
    order_rules,
    random_unitary,
    simulation)

from Scripts import simulation_CLI as cleo

In [3]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.2, 0.23, 0.36, 0.15, 0.18, 0.408125]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random


In [4]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.2, 0.23, 0.36, 0.15, 0.18, 0.408125]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random


In [5]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.2, 0.23, 0.36, 0.15, 0.18, 0.408125]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random


In [6]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random


In [7]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "random"
    num_steps = 500
    data = cleo.execute(file_name="random",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random
simulation complete, extracting and saving data to : ../data/random/random-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_random


In [8]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/random")

In [8]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [9]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [10]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [11]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [12]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "greedy"
    num_steps = 500
    data = cleo.execute(file_name="greedy",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy
simulation complete, extracting and saving data to : ../data/greedy/greedy-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_greedy


In [14]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/greedy")

In [13]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:56: RuntimeWarning: invalid value encountered in log
  return 1 / (np.log((1 - pop) / pop))
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: divide by zero encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)
/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in scalar multiply
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)


90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [14]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [15]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [16]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [17]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "mimic"
    num_steps = 500
    data = cleo.execute(file_name="mimic",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic
simulation complete, extracting and saving data to : ../data/mimic/mimic-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_mimic


In [20]:
import sys as SYS; SYS.path.insert(0, '../..')
from Scripts import hdf5merge
hdf5merge.merge_hdf5_files("../data/mimic")

In [18]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c2_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c2_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [19]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c4_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c4_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [20]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c5_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c5_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [21]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "c6_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

/Users/uja5020/Documents/GitHub/Qnibble/Notebooks/../src/measurements.py:74: RuntimeWarning: invalid value encountered in log
  tr_1 = (1 - pop_1) * np.log(1 - pop_1) + (pop_1) * np.log(pop_1)


90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-c6_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes


In [22]:
Trials = 1
for unitary_seed in range(Trials):
    #initial_pops = [0.31, 0.23, 0.22, .281714, 0.36, 0.15, 0.18, 0.23]
    #initial_pops = [0.5,0,0,0,0.5,0,0,0]
    initial_pops = [0.5,0,0,0,0.5,0,0,0]
    #initial_pops = [0.4, 0.2, .2, .2,0.4,0.2,0.2,0.2]
    system = DM.n_thermal_qbits(initial_pops)
    connectivity = "cN_2local"
    unitary_energy_subspace = 1
    ordering_seed = 1
    chunk_size = 2
    order_rule_name = "landscape_maximizes"
    num_steps = 500
    data = cleo.execute(file_name="landscape_maximizes",
                       connectivity=connectivity,
                       order_rule_name = order_rule_name, 
                       unitary_seed=unitary_seed,
                       unitary_energy_subspace=unitary_energy_subspace,
                       evolution_generation_type="haar2Qunitary",
                       chunk_size=chunk_size,
                       num_steps=num_steps,
                       initial_pops=initial_pops,
                       verbosity = .9)

90%
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
simulation complete, extracting and saving data to : ../data/landscape_maximizes/landscape_maximizes-8_qubits-cN_2local_connectivity-unitary_energy_subspace_1-unitary_seed_0-order_rule_name_landscape_maximizes
